# Colour Correction using Splines

### Define constants

In [18]:
%load_ext autoreload
%autoreload 2
import colour
from data import load_dataset_sfu, load_dataset_csv, load_cmfs, load_camera, load_insitu, msds_to_rgb, msds_to_xyz, load_dataset_skin
import numpy as np

RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)
    
SFU_FILE_PATH = 'data/reflect_db.reflect'
CAVE_FOSTER2004_PATH = 'data/cave_foster2002.csv'
INSITU_PATH = "data/insitu_dataset.csv"
CAVE_PATH = 'data/cave.csv'
FOSTER_50_PATH = 'data/foster50.csv'
CAMERA = 'sigma'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:

# sfu_dataset = load_dataset_sfu(SFU_FILE_PATH)
cave_foster2004_dataset = load_dataset_csv(CAVE_FOSTER2004_PATH)
foster_50_dataset = load_dataset_csv(FOSTER_50_PATH)
insitu = load_insitu(INSITU_PATH)
# cave = load_dataset_csv(CAVE_PATH)
# skin = load_dataset_skin()


In [20]:
from plotting import plot_chromaticity_diagram
TRAIN = insitu
TEST = foster_50_dataset
VALIDATION = insitu

### Computing Observer Responses
We can easily change the order of test and train sets here

In [21]:
from plotting import plot_chromaticity_diagram

cmfs, illuminant = load_cmfs()
response_trainset_xyz = msds_to_xyz(TRAIN, cmfs, illuminant)
response_testset_xyz = msds_to_xyz(TEST, cmfs, illuminant)
response_validation_xyz = msds_to_xyz(VALIDATION, cmfs, illuminant)

[ 0.94940092  1.          1.08709122]
[ 0.94940092  1.          1.08709122]
[ 0.94940092  1.          1.08709122]


### Computing Camera Responses

In [22]:
from utilities import add_noise
MSDS_TRAIN = load_camera(CAMERA)
response_trainset_camera = msds_to_rgb(TRAIN,MSDS_TRAIN, illuminant)
response_testset_camera = msds_to_rgb(TEST,MSDS_TRAIN, illuminant)
response_validationset_camera = msds_to_rgb(VALIDATION,MSDS_TRAIN, illuminant)
noise_gauss = 0.01
noise_poiss = 0.01
response_testset_camera_og = response_testset_camera
response_testset_camera = add_noise(response_testset_camera, noise_poiss, noise_gauss)

[  938.7258382   1714.98576086  1939.82119587]
[  938.7258382   1714.98576086  1939.82119587]
[  938.7258382   1714.98576086  1939.82119587]


### Fit Generalized Additive Model with P-splines

### Nikon

In [23]:
from sklearn.metrics import make_scorer
from colour_math import deltae_mean
from models import GAMOptimizer
from evaluate import pred
np.int = np.int64
if CAMERA == 'nikon':
    gam = GAMOptimizer(lams=1e-6, order=3, n_splines=5)
    
    # gam = GAMOptimizer(lams=0.0001,order=3, n_splines=10)
    # gam = GAMOptimizer(lams=0.01,order=3, n_splines=20)

    gam.fit(response_trainset_camera, response_trainset_xyz)

    pred(gam, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

### Sigma

In [24]:
from sklearn.metrics import make_scorer
from colour_math import deltae_mean
from models import GAMOptimizer
from evaluate import pred
np.int = np.int64
if CAMERA == 'sigma':
    # gam = GAMOptimizer(lams=1e-9, order=3, n_splines=5)
    # gam = GAMOptimizer(lams=0.0001,order=3, n_splines=10)
    gam = GAMOptimizer(lams=0.0001,order=3, n_splines=20)

    gam.fit(response_trainset_camera, response_trainset_xyz)

    pred(gam, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

3600
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 24.62
DeltaE max: 83.23
DeltaE median: 23.77
DeltaE 95 percentile: 47.70
DeltaE 99 percentile: 61.15


### Fit Linear Model

In [25]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression(fit_intercept=False)


linear.fit(response_trainset_camera, response_trainset_xyz)
pred(linear, response_testset_camera, response_testset_xyz, "Foster 50")


---- RESULTS Foster 50 ----
DeltaE mean: 25.23
DeltaE max: 92.83
DeltaE median: 23.83
DeltaE 95 percentile: 51.79
DeltaE 99 percentile: 66.75


In [26]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=1))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.9335701713566483
        x: [ 6.353e-01 -3.161e-02  3.444e-01 -9.674e-01  3.051e+00
            -1.087e+00  1.002e+00 -4.659e+00  4.749e+00]
      nit: 36
      jac: [ 2.205e-06  5.141e-07 -4.545e-07 -1.466e-05 -1.223e-05
            -1.052e-05  1.061e-05  9.581e-06  8.807e-06]
 hess_inv: [[ 1.112e+00 -2.620e+00 ..., -1.906e+00  1.129e+00]
            [-2.620e+00  6.452e+00 ...,  4.844e+00 -2.972e+00]
            ..., 
            [-1.906e+00  4.844e+00 ...,  8.027e+00 -5.041e+00]
            [ 1.129e+00 -2.972e+00 ..., -5.041e+00  3.255e+00]]
     nfev: 671
     njev: 66
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 24.61
DeltaE max: 91.25
DeltaE median: 23.25
DeltaE 95 percentile: 50.26
DeltaE 99 percentile: 65.13


### Fit 3rd order Root-Polynomial Model

In [27]:
from models import PolynomialTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])

RP_linear_3.fit(response_trainset_camera, response_trainset_xyz)

pred(RP_linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 34.19
DeltaE max: 146.45
DeltaE median: 24.35
DeltaE 95 percentile: 115.31
DeltaE 99 percentile: 129.86


### Fit a 2nd order Root-Polynomial Model

In [28]:
from models import GAMOptimizer, PolynomialTransformer, DeltaEOptimizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_2 = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])


RP_linear_2.fit(response_trainset_camera, response_trainset_xyz)
pred(RP_linear_2, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 25.31
DeltaE max: 105.88
DeltaE median: 23.74
DeltaE 95 percentile: 50.61
DeltaE 99 percentile: 66.51


In [29]:
P_Linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_Linear_3.fit(response_trainset_camera, response_trainset_xyz)
pred(P_Linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 24.87
DeltaE max: 85.64
DeltaE median: 23.93
DeltaE 95 percentile: 48.08
DeltaE 99 percentile: 62.26


In [30]:
P_linear = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_linear.fit(response_trainset_camera, response_trainset_xyz)
pred(P_linear, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 24.91
DeltaE max: 91.43
DeltaE median: 23.57
DeltaE 95 percentile: 50.51
DeltaE 99 percentile: 65.48


In [31]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000P = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=3))
])


DE2000P.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000P, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.7374502884923716
        x: [ 8.154e-01 -2.948e-01 ..., -4.671e-01 -1.659e+01]
      nit: 404
      jac: [ 2.249e-02  1.794e-02 ..., -4.389e-03 -3.579e-03]
 hess_inv: [[ 3.869e+00 -8.047e+00 ...,  2.658e+01 -9.805e+00]
            [-8.047e+00  1.931e+01 ..., -2.203e+02  3.724e+01]
            ..., 
            [ 2.658e+01 -2.203e+02 ...,  5.126e+04 -6.996e+03]
            [-9.806e+00  3.724e+01 ..., -6.996e+03  2.877e+03]]
     nfev: 27272
     njev: 470
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 24.31
DeltaE max: 127.62
DeltaE median: 23.61
DeltaE 95 percentile: 46.32
DeltaE 99 percentile: 58.40


In [32]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=True, degree=3))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.7326022046775705
        x: [ 1.137e+01 -1.398e+01 ..., -6.191e+00 -6.332e+01]
      nit: 200
      jac: [-4.732e-04 -3.758e-04 ..., -7.773e-04 -8.248e-04]
 hess_inv: [[ 1.927e+03  3.755e+03 ...,  4.703e+03  4.621e+03]
            [ 3.755e+03  8.841e+03 ...,  9.641e+03  1.094e+04]
            ..., 
            [ 4.703e+03  9.641e+03 ...,  2.025e+04  1.101e+04]
            [ 4.621e+03  1.094e+04 ...,  1.101e+04  1.443e+04]]
     nfev: 10092
     njev: 252
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 26.19
DeltaE max: 133.73
DeltaE median: 23.40
DeltaE 95 percentile: 58.13
DeltaE 99 percentile: 84.09
